In [5]:
!gcloud config set project "cloudcomputing-398719" #pick andyyang903
!gcloud auth application-default login


Updated property [core/project].
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=HtOf8R1tvDMcl9lEwe8ENyVyOCvqsp&access_type=offline&code_challenge=IuMX1ULF1Dl8Fd8JFnSx5VSwywbOK8LmUqBXk87_keM&code_challenge_method=S256


Credentials saved to file: [/Users/andyyang/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "cloudcomputing-398719" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [60]:

from google.cloud import storage
storage_client = storage.Client()

In [23]:
bucket = storage_client.bucket("ayang903-hw2-bucket")


In [19]:
# list all files
blobs = storage_client.list_blobs("ayang903-hw2-bucket", prefix="files/")

# read the files, 


In [61]:
blobs = storage_client.list_blobs("ayang903-hw2-bucket", prefix="files")
files = [blob.name for blob in blobs]

In [63]:
len(files)

10001

In [42]:
blob = bucket.blob("files/4.html")


In [43]:
content = blob.download_as_text()
content

'<!DOCTYPE html>\n<html>\n<body>\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad\nminim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse\ncillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n<p>\n<a HREF="9063.html"> This is a link </a>\n<p>\nLorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad\nminim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse\ncillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n<p>\n<a HREF="53

In [44]:
import re
len(re.findall(r'<a HREF="', content))

75

In [45]:
re.findall(r'<a HREF="(\d+.html)"', content)

['9063.html',
 '5357.html',
 '3015.html',
 '9716.html',
 '1323.html',
 '1680.html',
 '8736.html',
 '9516.html',
 '5042.html',
 '2562.html',
 '6170.html',
 '2407.html',
 '2051.html',
 '3650.html',
 '5175.html',
 '8328.html',
 '3978.html',
 '3877.html',
 '3013.html',
 '4768.html',
 '6102.html',
 '6877.html',
 '757.html',
 '2166.html',
 '9850.html',
 '336.html',
 '6452.html',
 '1276.html',
 '1199.html',
 '2163.html',
 '6885.html',
 '4905.html',
 '9023.html',
 '6828.html',
 '2331.html',
 '9684.html',
 '6917.html',
 '4882.html',
 '5809.html',
 '1385.html',
 '4064.html',
 '7287.html',
 '6049.html',
 '8670.html',
 '947.html',
 '6167.html',
 '6694.html',
 '138.html',
 '6835.html',
 '5253.html',
 '7230.html',
 '3342.html',
 '6088.html',
 '4805.html',
 '7715.html',
 '1491.html',
 '3038.html',
 '1779.html',
 '4537.html',
 '1837.html',
 '9145.html',
 '9921.html',
 '2521.html',
 '7309.html',
 '6533.html',
 '3037.html',
 '6909.html',
 '7073.html',
 '2862.html',
 '4062.html',
 '7429.html',
 '5576.htm

In [53]:
def read_file(bucket_name, file_name):
    """Read a file from GCS bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    return blob.download_as_text()

In [56]:
def get_stats(bucket_name):
    storage_client = storage.Client()
    blobs = storage_client.list_blobs("ayang903-hw2-bucket", prefix="files")
    files = [blob.name for blob in blobs]
    files = files[1:-1]

    incoming_links = {} # key: filename, value: count of incoming links
    outgoing_links = []

    for file in files:
            content = read_file(bucket_name, file)

            # Count outgoing links
            outgoing_count = len(re.findall(r'<a HREF="', content))
            outgoing_links.append(outgoing_count)

            # Update incoming links count for linked files
            for match in re.findall(r'<a HREF="(\d+.html)"', content):
                if match not in incoming_links:
                    incoming_links[match] = 0
                incoming_links[match] += 1
    # Create a list of incoming link counts, filling in 0 for files without any links
    file_names = [file.split('/')[-1] for file in files] # Get just the filenames without directory path
    incoming_counts = [incoming_links.get(file, 0) for file in file_names]
    
    # Calculate properties
    avg_incoming = sum(incoming_counts) / len(files)
    avg_outgoing = sum(outgoing_links) / len(files)
    
    median_incoming = sorted(incoming_counts)[len(files) // 2]
    median_outgoing = sorted(outgoing_links)[len(files) // 2]
    
    max_incoming = max(incoming_counts)
    max_outgoing = max(outgoing_links)

    min_incoming = min(incoming_counts)
    min_outgoing = min(outgoing_links)

    quintiles_incoming = [sorted(incoming_counts)[i * len(files) // 5] for i in range(1, 5)]
    quintiles_outgoing = [sorted(outgoing_links)[i * len(files) // 5] for i in range(1, 5)]
    
    return {
        "avg_incoming": avg_incoming,
        "avg_outgoing": avg_outgoing,
        "median_incoming": median_incoming,
        "median_outgoing": median_outgoing,
        "max_incoming": max_incoming,
        "max_outgoing": max_outgoing,
        "min_incoming": min_incoming,
        "min_outgoing": min_outgoing,
        "quintiles_incoming": quintiles_incoming,
        "quintiles_outgoing": quintiles_outgoing,
    }

In [57]:
get_stats('ayang903-hw2-bucket')

TransportError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [90]:
# !gcloud config set project "cloudcomputing-398719" #pick andyyang903
# !gcloud auth application-default login

from google.cloud import storage
import re
import numpy as np

# storage_client = storage.Client()



# def read_file(bucket_name, file_name):
#     """Read a file from GCS bucket."""
#     storage_client = storage.Client()
#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(file_name)
#     return blob.download_as_text()

def get_stats():
    storage_client = storage.Client()
    blobs = storage_client.list_blobs("ayang903-hw2-bucket", prefix="files")
    files = [blob.name for blob in blobs]
    files = files[1:]

    incoming_links = {} # key: filename, value: count of incoming links
    outgoing_links = []

    for file in files:
            bucket = storage_client.bucket("ayang903-hw2-bucket")
            blob = bucket.blob(file)
            content =  blob.download_as_text()
            # Count outgoing links
            outgoing_count = len(re.findall(r'<a HREF="', content))
            outgoing_links.append(outgoing_count)

            # Update incoming links count for linked files
            for match in re.findall(r'<a HREF="(\d+.html)"', content):
                if match not in incoming_links:
                    incoming_links[match] = 0
                incoming_links[match] += 1
    # Create a list of incoming link counts, filling in 0 for files without any links
    file_names = [file.split('/')[-1] for file in files] # Get just the filenames without directory path
    incoming_counts = [incoming_links.get(file, 0) for file in file_names]
    
    # Calculate properties
    avg_incoming = sum(incoming_counts) / len(files)
    avg_outgoing = sum(outgoing_links) / len(files)
    
    median_incoming = sorted(incoming_counts)[len(files) // 2]
    median_outgoing = sorted(outgoing_links)[len(files) // 2]
    
    max_incoming = max(incoming_counts)
    max_outgoing = max(outgoing_links)

    min_incoming = min(incoming_counts)
    min_outgoing = min(outgoing_links)

    quintiles_incoming = [sorted(incoming_counts)[i * len(files) // 5] for i in range(1, 5)]
    quintiles_outgoing = [sorted(outgoing_links)[i * len(files) // 5] for i in range(1, 5)]
    
    q1 =  {
        "avg_incoming": avg_incoming,
        "avg_outgoing": avg_outgoing,
        "median_incoming": median_incoming,
        "median_outgoing": median_outgoing,
        "max_incoming": max_incoming,
        "max_outgoing": max_outgoing,
        "min_incoming": min_incoming,
        "min_outgoing": min_outgoing,
        "quintiles_incoming": quintiles_incoming,
        "quintiles_outgoing": quintiles_outgoing,
    }

    return q1

def construct_graph():
    storage_client = storage.Client()
    blobs = storage_client.list_blobs("ayang903-hw2-bucket", prefix="files")
    files = [blob.name for blob in blobs]
    files = files[1:]

    graph = {} # key: filename, value: list of outgoing links

    for file in files:
        bucket = storage_client.bucket("ayang903-hw2-bucket")
        blob = bucket.blob(file)
        content =  blob.download_as_text()
        out_links = re.findall(r'<a HREF="(\d+.html)"', content)
        file_name_only = file.split('/')[-1]
        graph[file_name_only] = out_links

    return graph

# def pagerank_andy(page_links):
#     pages = list(page_links.keys())
#     N = len(pages)
#     d = 0.85
#     iterations = 0

#     # initial ranks
#     ranks = dict.fromkeys(pages, 1/N)
#     deltas = np.inf

#     while deltas > 0.005:
#         iterations += 1
#         new_rank = dict.fromkeys(pages, 0.15)
#         for page in pages:
#             for linked in page_links[page]:
#                 new_rank[linked] += d*ranks[page]/len(page_links[page])
#         deltas = sum(np.abs(new_rank[page] - ranks[page]) for page in pages)
#         ranks = new_rank
#     return ranks

def pagerank_dangle(page_links):
    pages = list(page_links.keys())
    N = len(pages)
    d = 0.85
    iterations = 0

    # Initial ranks
    ranks = dict.fromkeys(pages, 1/N)
    deltas = np.inf

    while deltas > 0.005:
        iterations += 1
        new_rank = dict.fromkeys(pages, 0)
        dangling_pr = 0

        # Handle dangling nodes
        for page in pages:
            if not page_links[page]:  # Check if page has no outgoing links (dangling)
                dangling_pr += ranks[page]

        # Distribute dangling PageRank equally among all pages
        for page in pages:
            new_rank[page] += d * dangling_pr / N

        # Calculate new ranks
        for page in pages:
            for linked in page_links[page]:
                new_rank[linked] += d * ranks[page] / len(page_links[page])

        # Adjust ranks with the random jump factor
        for page in pages:
            new_rank[page] = (1-d)/N + new_rank[page]

        # Calculate the change in ranks for convergence check
        deltas = sum(np.abs(new_rank[page] - ranks[page]) for page in pages)
        ranks = new_rank

    print(f"Converged in {iterations} iterations")
    return ranks

# stats = get_stats()
# print(stats)
# graph = construct_graph()

In [107]:
def compute_pagerank(graph):
    num_nodes = len(graph)
    pr = {node: 55.9/num_nodes for node in graph}  # initialize all nodes with uniform PR
    
    while True:
        next_pr = {node: 0 for node in graph}  # initialize next_pr
        dangling_pr = 0
        
        # Handle dangling nodes and distribute dangling PageRank equally among all nodes
        for node in graph:
            if not graph[node]:  # if node has no outgoing links (dangling)
                dangling_pr += pr[node]
        
        for node in graph:
            next_pr[node] += 0.85 * dangling_pr / num_nodes
        
        # find the next pageranks
        for node in graph:
            for out_link in graph[node]:
                next_pr[out_link] += 0.85 * pr[node] / len(graph[node])
            
        # Adjust ranks with the random jump factor
        for node in graph:
            next_pr[node] = 0.15 + next_pr[node]

        # Check for convergence based on the absolute differences in pageranks
        deltas = sum(abs(next_pr[node] - pr[node]) for node in graph)
        if deltas < 0.005:
            break

        pr = next_pr
    return pr


In [112]:
## TEMP

def compute_pagerank(graph):
    num_nodes = len(graph)
    pr = {node: 55.9/num_nodes for node in graph}  # initialize all nodes with uniform PR
    
    while True:
        sum_before_iteration = sum(pr.values())

        next_pr = {node: 0 for node in graph}  # initialize next_pr
        dangling_pr = 0
        
        # Handle dangling nodes and distribute dangling PageRank equally among all nodes
        for node in graph:
            if not graph[node]:  # if node has no outgoing links (dangling)
                dangling_pr += pr[node]
        
        for node in graph:
            next_pr[node] += 0.85 * dangling_pr / num_nodes
        
        # find the next pageranks
        for node in graph:
            for out_link in graph[node]:
                next_pr[out_link] += 0.85 * pr[node] / len(graph[node])
            
        for node in graph:
            next_pr[node] = 0.15 + next_pr[node]

        sum_after_iteration = sum(next_pr.values())


        # Check for convergence 
        if abs((sum_after_iteration - sum_before_iteration)/abs(sum_before_iteration)) < 0.005:
            break

        pr = next_pr
    return pr


In [88]:
graph = construct_graph()

In [113]:
pr = compute_pagerank(graph)
top_5_pages = sorted(pr, key=pr.get, reverse=True)[:5]
for page in top_5_pages:
    print(f"{page}: {pr[page]}")


2526.html: 2.305310514788749
6846.html: 2.080249091026444
5971.html: 2.0284341876521963
5778.html: 2.0176445847265496
4961.html: 2.015012345829614


In [ ]:
2526.html: 2.3195265132379252
6846.html: 2.090961580940669
5971.html: 2.0380346664859954
5778.html: 2.0267191109765412
4961.html: 2.0240055239291417

# 5453.html: 5.5517334959965225
# 7853.html: 5.4207740317221
# 6991.html: 5.417067232118919
# 2451.html: 5.410213524773686
# 8090.html: 5.405786885984594

In [ ]:
#v1
for node in graph:
            # Compute the sum of PR(Ti)/C(Ti), skip if C(Ti) is 0
            sum_pr_over_c = sum(pr[out_link] / len(graph[out_link]) for out_link in graph[node] if out_link in graph and len(graph[out_link]) > 0)
            next_pr[node] = (1-d) + d * sum_pr_over_c

#v2
for node in graph:
            # Compute incoming rank from other nodes
            incoming_rank_sum = sum(pr[out_link] / len(graph[out_link]) if len(graph[out_link]) > 0 else 0 for out_link in graph if node in graph[out_link])
            
            # Apply the PageRank formula
            next_pr[node] = (1-d) / num_nodes + d * incoming_rank_sum
        